<a href="https://colab.research.google.com/github/giulianogrossi/imgs/blob/main/pyVHR_run_on_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **RUN on dataset**
---

In [5]:
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.analysis.stats import StatAnalysis
from pyVHR.plot.visualize import *
import os
import pandas as pd
import plotly.express as px
from numpy import round as r

# main functions 

def ANALYSIS(dataset, approach, metrics=['MAE'], j_data=False, writeFile=False):
  """Print boxplot analysis"""

  cd = os.getcwd() # current dir
  if not j_data:
    print('## Dataset: ' + dataset)
    print('   Approach: ' + approach)
    path = path_results + dataset + "_" + approach + ".h5"
    filenameH5 = os.path.join(cd, path)
    print('   h5      :', path)
    cfg = path_results + dataset + "_" + approach + ".cfg"
    print('   cfg     :', cfg, end='\n\n')
  else:
    print('## Path dir: ' + dataset)
    print('   Approach: ' + approach)
    path = path_results + dataset 
    filenameH5 = os.path.join(cd, path)
    print('   path  = ', path)

  # boxplot
  for m in metrics:
    st = StatAnalysis(filenameH5, join_data=j_data)
    if m=='MAE': 
      scale = 'log' 
    else: 
      scale = 'linear'
    fig = st.displayBoxPlot(metric=m, scale=scale)
    fig.show(renderer = "colab")
    if writeFile:
      filenamePNG = path_results + dataset + "_" + approach + "_" + m + "_MAE.png"
      fig.write_image(filenamePNG)  

def STATS(dataset, approach='clustering'):
  """Do stats analysis"""
  
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + approach + ".h5"
  df = pd.read_hdf(os.path.join(cd, path))

  # compute errors
  MAE_mean, MAE_std, MAE_median =    r(df['MAE'].mean()[0],2),  r(df['MAE'].std(),2),  r(df['MAE'].median(),2)
  RMSE_mean, RMSE_std, RMSE_median = r(df['RMSE'].mean()[0],2), r(df['RMSE'].std(),2), r(df['RMSE'].median(),2)
  PCC_mean, PCC_std, PCC_median =    r(df['PCC'].mean()[0],2),  r(df['PCC'].std(),2),  r(df['PCC'].median(),2)
  CCC_mean, CCC_std, CCC_median =    r(df['CCC'].mean()[0],2),  r(df['CCC'].std(),2),  r(df['CCC'].median(),2)
  SNR_mean, SNR_std, SNR_median =    r(df['SNR'].mean()[0],2),  r(df['SNR'].std(),2),  r(df['SNR'].median(),2)
  MAX_mean, MAX_std, MAX_median =    r(df['MAX'].mean()[0],2),  r(df['MAX'].std(),2),  r(df['MAX'].median(),2)

 # print table 
  table = [[MAE_mean, MAE_std, MAE_median], [RMSE_mean, RMSE_std, RMSE_median], [PCC_mean, PCC_std, PCC_median],
           [CCC_mean, CCC_std, CCC_median], [SNR_mean, SNR_std, SNR_median], [MAX_mean, MAX_std, MAX_median]]
  stats = pd.DataFrame(table, columns = [ 'MEANS', 'STD', 'MEDIANS'], index=['MAE', 'RMSE', 'PCC', 'CCC', 'SNR', 'MAX'])
  print('\n** Stats table **')
  print(stats)
					
def RUN(dataset, method):
  """Run methods on a dataset"""

  # set paths
  print('## Dataset: ' + dataset)
  print('Using...')
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + method + ".h5"
  print('   filename h5: ', path)
  cfg = path_results + dataset + "_" + method + ".cfg"
  print('   cfg        :', cfg)
  print('\n')

  # pipeline
  pl = Pipeline()
  res = pl.run_on_dataset(os.path.join(cd, cfg), verb=1)
  res.saveResults(filenameH5)
  print('Written file: ' + filenameH5 + '\n\n')

def DETAILS(dataset, method, metric='MAE'):
  """Show performance details on each video of the dataset"""

  # set paths
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + method + ".h5"
  print('## Dataset: ' + dataset)
  print('   filename h5: ', path)
  df = pd.read_hdf(os.path.join(cd, path))
  datasetname = df['dataset'][0].upper()
  methods = set(list(df['method']))

  # loop on methods
  for m in methods:
    print('   method     :', m)
    vals = df[df['method'] == m][metric]
    fvideos = df[df['method'] == m]['videoFilename']
    x = []
    y = []
    for i in vals.index:
      n = fvideos[i]
      n = n[n.rfind('/', 0, n.rfind('/'))+1:n.rfind('/')]
      x.append(n)
      y.append(vals[i][0])
    fig = px.bar(x=x, y=y, title='dataset: ' + datasetname + ' method: ' + m)
    fig.update_xaxes(type='category')
    fig.show(renderer = "colab")


This notebook shows the complete **pipeline** that takes a **dataset** as input and computes an **estimation** of **BPMs** using a prespecified set of **rPPG** **methods**.

Below are the possible choices for the methods.
* **Methods**: `cpu_CHROM`, `cupy_CHROM`, `torch_CHROM`, `cpu_LGI`, `cpu_POS`, `cupy_POS`, `cpu_PBV`, `cpu_PCA`, `cpu_GREEN`, `cpu_OMIT`, `cpu_ICA`, `cpu_SSR`

Below are the possible choices for the datasets.
* **Datasets**: `PURE`, `UBFC1`, `UBFC2`, `ECG_Fitness_01-1`, `ECG_Fitness_01-2`... `ECG_Fitness_06-1`, `ECG_Fitness_06-2`

#Runs

In [16]:
# run on dataset

path_results = "results/cfg/" # general path for cfg

#dataset = 'PURE'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'UBFC1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'UBFC2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'LGI_PPGI'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_01-1'   FIN
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_01-2'   
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_02-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_02-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_03-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_03-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_04-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_04-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_05-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_05-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_06-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_06-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

# Analisys

In [6]:
## -- Holistic

path_results = "results/h5/" # general path for cfg# general path for data

dataset = 'UBFC2'
ANALYSIS(dataset, 'holistic', metrics=['MAE'])
STATS(dataset, approach='holistic')

## Dataset: UBFC2
   Approach: holistic
   h5      : results/h5/UBFC2_holistic.h5
   cfg     : results/h5/UBFC2_holistic.cfg




** Stats table **
      MEANS    STD  MEDIANS
MAE   10.55  11.42     5.84
RMSE  16.87  15.93    13.61
PCC    0.47   0.45     0.42
CCC    0.43   0.43     0.24
SNR    2.85   4.86     2.19
MAX   48.74  41.33    49.10


In [7]:
## -- Median
ANALYSIS(dataset, 'median', metrics=['MAE'])
STATS(dataset, approach='median')

## Dataset: UBFC2
   Approach: median
   h5      : results/h5/UBFC2_median.h5
   cfg     : results/h5/UBFC2_median.cfg




** Stats table **
      MEANS    STD  MEDIANS
MAE    5.27  12.99     1.88
RMSE   8.27  14.96     2.95
PCC    0.70   0.35     0.90
CCC    0.64   0.36     0.83
SNR   -0.37   2.65    -0.37
MAX   24.94  29.57    13.65


In [11]:
## -- Clustering
ANALYSIS(dataset, 'clustering', metrics=['MAE'])
STATS(dataset, approach='clustering')

## Dataset: UBFC2
   Approach: clustering
   h5      : results/h5/UBFC2_clustering.h5
   cfg     : results/h5/UBFC2_clustering.cfg




** Stats table **
      MEANS    STD  MEDIANS
MAE    3.61  13.24     1.17
RMSE   5.25  14.84     1.45
PCC    0.84   0.28     0.96
CCC    0.81   0.27     0.93
SNR   -0.37   2.65    -0.35
MAX   14.64  26.49     3.65


In [12]:
## -- Deep 
ANALYSIS(dataset, 'deep', metrics=['MAE'])
STATS(dataset, approach='deep')

## Dataset: UBFC2
   Approach: deep
   h5      : results/h5/UBFC2_deep.h5
   cfg     : results/h5/UBFC2_deep.cfg




** Stats table **
      MEANS    STD  MEDIANS
MAE   19.00  15.09    15.82
RMSE  25.12  16.00    23.94
PCC    0.14   0.38     0.05
CCC    0.13   0.28     0.01
SNR   -1.99   4.11    -1.97
MAX   52.18  25.60    54.57


#Details

In [13]:
# Analysis - holistic

method = 'holistic'
DETAILS(dataset, method, metric='MAE')

## Dataset: UBFC2
   filename h5:  results/h5/UBFC2_holistic.h5
   method     : ICA


   method     : POS


   method     : PCA


   method     : LGI


   method     : CHROM


In [15]:
# Analysis - median
metric='MAE'
method = 'median'
DETAILS(dataset, method, metric)

## Dataset: UBFC2
   filename h5:  results/h5/UBFC2_median.h5
   method     : ICA


   method     : POS


   method     : PCA


   method     : LGI


   method     : CHROM


In [16]:
# Analysis - clustering
metric='MAE'
method = 'clustering'
DETAILS(dataset, method, metric)

## Dataset: UBFC2
   filename h5:  results/h5/UBFC2_clustering.h5
   method     : ICA


   method     : POS


   method     : PCA


   method     : LGI


   method     : CHROM


# Multidataset

In [9]:
# multi dataset
#path = path_results + 'h5_collection'
#st = StatAnalysis(path, join_data=True)
#fig = st.displayBoxPlot(metric='PCC')
#fig.show(renderer = "colab")
